<a href="https://colab.research.google.com/github/KennyNg-19/colab_nb/blob/main/CS4243.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS4243
## Image Classification

### Three classes: normal, carrying, threat


Kenny Ng: MLP+CNN

Junzhe: CNN+RNN

Qini: VGG + resnet18 + LeNet

11.7 周一 - preprocessing + 开始构建模型、

11.8 周二 - 模型可以跑通

周三 - eval 完成 + 讨论需不需要做更多 + 查漏补缺

周四 - 写poster + 查漏补缺

周五 - 讨论如何presentation + 查漏补缺

周六 - presentation

需要交的东西：

poster，模型weights


# TODO:
1. need to somehow load data and potentially use dataloader to split batch
2. a DF ideally contains file names and label

In [13]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/cs4243_smallest
!ls

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/cs4243_smallest
carrying  normal  threat


### Import Library

In [18]:
import os
from PIL import Image
import cv2

import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

from random import randint

from tqdm import tqdm

import time

In [3]:
# gpu setting
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


### Helper Functions

In [4]:
# Class to save the best model while training - "best_model"
class SaveBestModel:
    def __init__(
        self, best_valid_loss=float('inf')
    ):
        self.best_valid_loss = best_valid_loss
        
    def __call__(
        self, current_valid_loss, 
        epoch, model, optimizer, criterion
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            print(f"\nBest validation loss: {self.best_valid_loss}")
            print(f"\nSaving best model for epoch: {epoch+1}\n")
            torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, 'outputs/best_model.pth')
            
def save_model(epochs, model, optimizer, criterion):
    print(f"Saving final model...")
    torch.save({
                'epoch': epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, 'outputs/final_model.pth')

In [5]:
# more info for the poster:
plt.style.use('ggplot')

def save_plots(train_acc, valid_acc, train_loss, valid_loss):
    # accuracy
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_acc, color='green', linestyle='-', 
        label='train accuracy'
    )
    plt.plot(
        valid_acc, color='blue', linestyle='-', 
        label='validataion accuracy'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('outputs/accuracy.png')
    
    # loss
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_loss, color='orange', linestyle='-', 
        label='train loss'
    )
    plt.plot(
        valid_loss, color='red', linestyle='-', 
        label='validataion loss'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('outputs/loss.png')

def creat_viz_cm(confusion_matrix, label2class):
    plt.figure(figsize=(15,10))

    class_names = list(label2class.values())
    df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=15)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=15)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


### Loading the image and preprocessing
Scaling and cropping, image enhancement, transformation to Tensor, normalization

从输入到最后一个卷积特征feature map，就是进行信息抽象的过程，然后就经过全连接层/全局池化层的变换进行分类了，这个feature map的大小，可以是3*3，5*5，7*7等等。在这些尺寸中，如果尺寸太小，那么信息就丢失太严重，如果尺寸太大，信息的抽象层次不够高，计算量也更大，所以7*7的kernel大小是一个最好的平衡。

另一方面，图像从大分辨率降低到小分辨率，降低倍数通常是2的指数次方，所以图像的输入是 kernel size *2的指数次方。

In [6]:
#labels
classes_mapping = {'normal':0, 'carrying':1, 'threat':2}
classes = list(classes_mapping.keys()) # 保持有序
class_int = list(classes_mapping.values())
classes, class_int


(['normal', 'carrying', 'threat'], [0, 1, 2])

#### generate DataFrame for whole dataset

In [19]:

img_names = [] # ID
targets = [] # labels
# root_path='.'
root_path = "/content/drive/My Drive/Colab Notebooks/cs4243_smallest"
# root_path = "/content/drive/My Drive/Colab Notebooks/cs4243_smallest"

# Mapping all img names & labels, 也是按照 classes顺序， ['normal', 'carrying', 'threat'] [0,1,2]
for class_path in classes:                    
    
    # print(class_path)
    path = os.path.join(root_path, class_path)
    # 加上label_class_path
    # img_names += [ class_path + "/" + img_name for img_name in os.listdir(path)]
    print(class_path, len(os.listdir(path)))
    img_names += os.listdir(path)
    # print(os.listdir(self.path))
    targets += [classes_mapping[class_path]] * len(os.listdir(path)) # all labels = 0 or 1 or 2

# idx 完全mapping
assert len(img_names) == len(targets), f'wrong no of all imgs, {len(img_names)}, {len(targets)}'
df = pd.DataFrame({'ID':img_names, 'Label': targets})
df.head()


normal 1265
carrying 1535
threat 1052


,ID,Label
0,A0227648U_normal_44440.29563_100.png,0
1,a015032m_040922_normal_13440.34438_30.png,0
2,0218134_20220904_82510.38845_200.png,0
3,normal 20.10089_30.png,0
4,1054350_20220904_normal_00040.37869_100.png,0


In [21]:
# split into df of train / test 
y = df['Label']
X = df

train_df, test_df, train_label, test_label = train_test_split(X, y, stratify=y, shuffle=True, test_size=0.2) 
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

# torch.utils.data.random_split只是随机分，没有按照类别Stratified，比例：1857 ： 1535 ：1551
# train_df, test_df = torch.utils.data.random_split(dataset, [train_size, test_size])

# check Stratified split, 的class分布
print(train_df.Label.value_counts()/train_df.Label.value_counts().sum())
test_df.Label.value_counts()/test_df.Label.value_counts().sum()


1    0.398572
0    0.328465
2    0.272963
Name: Label, dtype: float64


1    0.398184
0    0.328145
2    0.273671
Name: Label, dtype: float64

#### Image preporcess: transform

In [ ]:
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
normalize = transforms.Normalize(mean, std)

resize_H, resized_W = 7*2**5, 7*2**5 # 如果是toTensor前，最后一个transform操作(即输入到模型的input shape)，最好是kerel size, 3/5/7,* 2的指数次方
resize = transforms.Resize([resize_H, resized_W])
# resize = transforms.Resize([resize_H, resized_W],interpolation=torchvision.transforms.InterpolationMode.BILINEAR)

crop_H, crop_W = 7*2**5, 7*2**5 # 如果是toTensor前，最后一个transform操作(即输入到模型的input shape)，最好是kerel size, 3/5/7,* 2的指数次方
cent_crop = transforms.CenterCrop([crop_H, crop_W])

transformations = transforms.Compose([resize,
                                    #  cent_crop,
                                    transforms.ToTensor(), # Tturn gray level from 0-255 into 0-1
                                    normalize])  #  change 0-1 into (-1, 1)



#### Build Dataset & DataLoader

In [ ]:
class CS4243_dataset(Dataset): 
    
    
    def __init__(self, root_path , dataframe, transform=None):
        
        self.classes_mapping = {0:'normal', 1:'carrying', 2:'threat'}
        self.df = dataframe    
        self.transform = transform
        self.root_path = root_path
        
        self.image_paths = self.df['ID'] #image names
        self.labels = self.df['Label']
                

    def __getitem__(self, index):
        
        img_path = self.image_paths[index] # 没有class_file_path
        # print(img_path)
        class_path = self.classes_mapping[self.labels[index]]
        image = Image.open(os.path.join(self.root_path, class_path, img_path))
        
        target = torch.tensor(self.labels[index])
      
        if self.transform != None:
            image = self.transform(image)
          
        return [image, target]
       
    def __len__(self):
        return len(self.df)

In [ ]:
batch_size = 64



train_dataset = CS4243_dataset(root_path, train_df, transform = transformations)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=2, shuffle = True, pin_memory=True)

test_dataset = CS4243_dataset(root_path, test_df, transform = transformations)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=2, shuffle = True, pin_memory=True)


for data in train_loader:
    imgs, targets = data
    print(imgs.shape)
    print('labels:', targets, len(targets))
    # 只输出一次
    break

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


NameError: ignored

### Image Classification Models

Implement image classificaiton by using:
1. VGG as baseline
2. Resnet18
3. Transfer Learning


#### Baseline: MLP

In [ ]:
# MLP

#### Way 1: LeNet from scratch

In [ ]:
class LeNet5_convnet(nn.Module):

    def __init__(self):

        super(LeNet5_convnet, self).__init__()
        self.conv1 = nn.Conv2d(1,   50,  kernel_size=3,  padding=1 )
        self.pool1  = nn.MaxPool2d(2,2)

        self.conv2 = nn.Conv2d(50,  100,  kernel_size=3,  padding=1 )
        self.pool2 = nn.MaxPool2d(2,2)
    
        self.linear1 = nn.Linear(4900, 100)
        self.linear2 = nn.Linear(100,10)


    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
    
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool2(x)

        x = x.view(-1, 4900)
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
    
        return x

#### Way 2: VGG from scratch
Build a VGG

In [ ]:
class VGG_convnet(nn.Module):

    def __init__(self):

        super(VGG_convnet, self).__init__()
     
        self.conv1a = nn.Conv2d(3,   64,  kernel_size=3, padding=1 )
        self.conv1b = nn.Conv2d(64,  64,  kernel_size=3, padding=1 )
        self.pool1  = nn.MaxPool2d(2,2)

        self.conv2a = nn.Conv2d(64,  128, kernel_size=3, padding=1 )
        self.conv2b = nn.Conv2d(128, 128, kernel_size=3, padding=1 )
        self.pool2  = nn.MaxPool2d(2,2)
     
        self.conv3a = nn.Conv2d(128, 256, kernel_size=3, padding=1 )
        self.conv3b = nn.Conv2d(256, 256, kernel_size=3, padding=1 )
        self.pool3  = nn.MaxPool2d(2,2)
        
        self.conv4a = nn.Conv2d(256, 512, kernel_size=3, padding=1 )
        self.pool4  = nn.MaxPool2d(2,2)

        # linear:   512 x 2 x 2 --> 2048 --> 4096 --> 4096 --> 10
        self.linear1 = nn.Linear(2048, 4096)
        self.linear2 = nn.Linear(4096,4096)
        self.linear3 = nn.Linear(4096, 10)


    def forward(self, x):

        x = self.conv1a(x)
        x = F.relu(x)
        x = self.conv1b(x)
        x = F.relu(x)
        x = self.pool1(x)

        x = self.conv2a(x)
        x = F.relu(x)
        x = self.conv2b(x)
        x = F.relu(x)
        x = self.pool2(x)

        x = self.conv3a(x)
        x = F.relu(x)
        x = self.conv3b(x)
        x = F.relu(x)
        x = self.pool3(x)

        x = self.conv4a(x)
        x = F.relu(x)
        x = self.pool4(x)

        # linear:   512 x 2 x 2 --> 2048 --> 4096 --> 4096 --> 10
        x = x.view(-1, 2048)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x) 
        
        return x

#### Way 3: Resnet 18

In [ ]:
class Residual_block(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(Residual_block, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
            
    def forward(self, x):
        out = self.left(x)
        out = out + self.shortcut(x)
        out = F.relu(out)
        
        return out

class ResNet(nn.Module):
    def __init__(self, Residual_block, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self.make_layer(Residual_block, 64, 2, stride=1)
        self.layer2 = self.make_layer(Residual_block, 128, 2, stride=2)
        self.layer3 = self.make_layer(Residual_block, 256, 2, stride=2)        
        self.layer4 = self.make_layer(Residual_block, 512, 2, stride=2)        
        self.fc = nn.Linear(512, num_classes)
        
    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def ResNet18():
    return ResNet(Residual_block)

## Train

In [ ]:
net=VGG_convnet()
# net=LeNet5_convnet()
# net = MLP()
# net = ResNet18()

print(net)

VGG_convnet(
  (conv1a): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2a): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2b): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3a): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3b): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=2048, out_features=4096, bias=True)
  (linear2): Linear(i

In [ ]:
# send to gpu
net = net.to(device)
mean = torch.FloatTensor(mean)
mean = mean.to(device)
std = torch.FloatTensor(std)
std = std.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
my_lr=0.25 
bs= 128
epochs = 1

# initialize SaveBestModel class
save_best_model = SaveBestModel()

In [ ]:
def train(model, train_loader, optimizer, criterion):
    model.train()
    print('===== Training =====')
    running_loss=0
    counter=0

    for i, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        counter += 1
        X_train, y_train = data
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        optimizer.zero_grad()

        # forward pass
        outputs = model(X_train)
        # calculate the loss
        loss = criterion(outputs, y_train)
        running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs.data, 1)
        running_correct += (preds == y_train).sum().item()
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()
        
        # loss and accuracy for the complete epoch
        epoch_loss = running_loss / counter
        epoch_acc = 100. * (running_correct / len(train_loader.dataset))
    return epoch_loss, epoch_acc
  

In [ ]:
def validate(model, test_loader, criterion):
    model.eval() # close BN and dropout layer
    print('===== Validation =====')
    running_loss = 0.0
    running_correct = 0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_loader), total=len(test_loader)):
            counter += 1
            
            X_test, y_test = data
            X_test = X_test.to(device)
            y_test = y_test.to(device)
            # forward pass
            outputs = model(X_test)
            # calculate the loss
            loss = criterion(outputs, y_test)
            running_loss += loss.item()
            # calculate the accuracy
            _, preds = torch.max(outputs.data, 1)
            running_correct += (preds == y_test).sum().item()
        
    # loss and accuracy for the complete epoch
    epoch_loss = running_loss / counter
    epoch_acc = 100. * (running_correct / len(test_loader.dataset))
    return epoch_loss, epoch_acc

In [ ]:
start=time.time()
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []
for epoch in range(epochs):
    print(f"[INFO]: Epoch {epoch+1} of {epochs}")
    
    # divide the learning rate by 2 at epoch 10, 14 and 18
    if epoch==10 or epoch == 14 or epoch==18:
        my_lr = my_lr / 2
    
    # give the current learning rate for the optimizer
    optimizer=torch.optim.SGD(net.parameters(), lr=my_lr, momentum=0.9, weight_decay=5e-4 )
        
    train_epoch_loss, train_epoch_acc = train(net, train_loader, 
                                            optimizer, criterion)
    valid_epoch_loss, valid_epoch_acc = validate(net, valid_loader,  
                                                criterion)
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")
    # save the best net:
    save_best_model(
        valid_epoch_loss, epoch, net, optimizer, criterion
    )
    print('-'*50)
    
# save the trained net weights for a final time - need to submit this with the poster
save_model(epochs, net, optimizer, criterion)
save_plots(train_acc, valid_acc, train_loss, valid_loss)
print('TRAINING COMPLETE')
    

## Prediction / Evaluation 

Evaluation by using:
1. accurancy, precision and recall (PR curve)
2. confusion matrix (visualised)
3. ROC (optional)

In [ ]:
# load the best model checkpoint
best_model_cp = torch.load('outputs/best_model.pth')
best_model_epoch = best_model_cp['epoch']
print(f"Best model was saved at {best_model_epoch} epochs\n")

# load the last model checkpoint
last_model_cp = torch.load('outputs/final_model.pth')
last_model_epoch = last_model_cp['epoch']
print(f"Last model was saved at {last_model_epoch} epochs\n")

In [ ]:
def test(model, test_loader):
    model.eval()
    print('===== Testing =====')
    valid_running_correct = 0
    counter = 0
    nb_classes = 3
    confusion_matrix = torch.zeros(nb_classes, nb_classes)

    with torch.no_grad():
        for i, data in tqdm(enumerate(test_loader), total=len(test_loader)):
            counter += 1
            
            X_test, y_test = data
            X_test = X_test.to(device)
            y_test = y_test.to(device)
            # forward pass
            outputs = model(X_test)

            # calculate the accuracy
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == y_test).sum().item()
            
            # get true and predict:
            for t, p in zip(y_test.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
        
    # loss and accuracy for the complete epoch
    final_acc = 100. * (valid_running_correct / len(test_loader.dataset))
    return final_acc, confusion_matrix

In [ ]:
# test two ckpts

def test_last_model(model, checkpoint, test_loader):
    print('Loading last epoch saved model weights...')
    model.load_state_dict(checkpoint['model_state_dict'])
    test_acc, confusion_matrix = test(model, test_loader)
    print(f"Last epoch saved model accuracy: {test_acc:.3f}")
    creat_viz_cm(confusion_matrix, label2class)

# test the best epoch saved model
def test_best_model(model, checkpoint, test_loader):
    print('Loading best epoch saved model weights...')
    model.load_state_dict(checkpoint['model_state_dict'])
    test_acc, confusion_matrix = test(model, test_loader)
    print(f"Best epoch saved model accuracy: {test_acc:.3f}")
    creat_viz_cm(confusion_matrix, label2class)

In [ ]:
# run test
test_last_model(net, last_model_cp, test_loader)

test_best_model(net, best_model_cp, test_loader)

#### Comapre with models in torchvision (resnet18 etc...)

In [ ]:
from torchvision import models

In [ ]:
model = models.resnet18(pretrained=True) 

# model = models.resnet152(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
model = model.eval()
model = model.to(device)